In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
file_id = '130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7'
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
df = pd.read_csv(download_url)

In [ ]:
# Функция для извлечения и сложения чисел из скобок
def resum(text):
    if pd.isna(text):  # Если NaN
        return 0
    # Ищем все числа в скобках и суммируем их
    numbers = re.findall(r'\((\d+)\)', str(text))
    return sum(map(int, numbers)) if numbers else 0

In [ ]:
#Как быстро избавиться от ненужного в df

#Выдворяем иногентов, убираем всё что не в рублях
rub_mask = df['Цена'].str.contains(' руб.', na=False)
cleared = df[rub_mask].copy()

# Оставляем только Москву, убираем другие регионы
msk_mask = cleared['Адрес'].str.contains('осква', na=False)
cleared = cleared[msk_mask].copy()

#Вычисляем общую площадь квартиры
cleared['total_area'] = cleared['Площадь, м2'].str.split('/').str[0].astype('float64')

#Вытаскиваем из колонки 'Цена' цену за аренду в месяц. Вычисляем цену аренды за метр квадратный и делаем флаг, который показывает включены ли платежи в стоимость аренды
cleared['price'] = cleared['Цена'].str.split(' ').str[0].str.strip().astype('float64')
cleared['price_per_sqm'] = cleared['price'] / cleared['total_area']
cleared['utility_payments'] = cleared['Цена'].str.contains('Коммунальные услуги включены', na=False).astype('int8')

# Преобразуем столбец 'Количество комнат' в 'rooms_count' и 'rooms_layout'
cleared['rooms_count'] = cleared['Количество комнат'].str.split(',').str[0].fillna('0').astype('int8')
cleared['rooms_layout'] = cleared['Количество комнат'].fillna('0').str.split(',').str[1].str.strip().str.lower()
cleared.loc[cleared['Количество комнат'].isna(),'rooms_layout'] = 'студия'
cleared['rooms_layout'] = cleared['rooms_layout'].fillna('unknown')

#Разбиваем на категории тип планировки, чтобы было проще кодировать дальше
cats = ['студия','unknown','изолированная','смежная','оба варианта']
cleared['rooms_layout'] = pd.Categorical(cleared['rooms_layout'], categories=cats)

# Делаем флаги на планировки
cleared['has_layout'] = cleared['rooms_layout'].isin(['смежная', 'изолированная', 'оба варианта']).astype('int8')
cleared['layout_is_adjacent'] = cleared['rooms_layout'].isin(['смежная', 'оба варианта']).astype('int8')
cleared['layout_is_isolated'] = cleared['rooms_layout'].isin(['изолированная', 'оба варианта']).astype('int8')
cleared['layout_is_studio'] = cleared['rooms_layout'].eq('студия').astype('int8')
cleared['layout_is_unknown'] = cleared['rooms_layout'].eq('unknown').astype('int8')

# Готовим столбец 'Парковка'. Преобразуем категории, чтоб их было меньше, было проще закодировать
cleared['parking_type'] = cleared['Парковка'].fillna('unknown')
cleared['parking_is_unknown'] = cleared['parking_type'].eq('unknown').astype('int8')
parking_map = {'unknown':'наземная', 'наземная':'наземная', 'подземная':'подземная', 'открытая':'наземная', 'многоуровневая':'подземная', 'на крыше':'наземная'}
cleared['parking_type'] = cleared['parking_type'].map(parking_map)
parking_cats = ['наземная','подземная']
cleared['parking_type'] = cleared['parking_type'].eq('подземная').astype('int8')


#Переделываем столбец 'Ремонт'. Разбиваем по категориям
cleared['renovation'] = cleared['Ремонт'].fillna('unknown')
renovation_cats = ['Без ремонта', 'Косметический', 'Евроремонт', 'unknown', 'Дизайнерский']
cleared['renovation'] = pd.Categorical(cleared['renovation'], categories=renovation_cats)

#Делаем категориям грейд по их уровню по дороговизне
renovation_grade = {'Без ремонта':0, 'Косметический':1, 'Евроремонт':2, 'Дизайнерский':3}
cleared['renovation_grades'] = cleared['renovation'].map(renovation_grade).astype('Int8')

#Делаем флаги есть ли ремонт
cleared['is_renovated'] = cleared['renovation'].isin(['Косметический', 'Евроремонт', 'Дизайнерский']).astype('int8')
cleared['renovation_is_unknown'] = cleared['renovation'].eq('unknown').astype('int8')


#делаем что-то с Мусоропроводом
cleared['trash'] = cleared['Мусоропровод'].fillna('unknown')
trash_cats = ['Да', 'Нет', 'unknown']
cleared['trash'] = pd.Categorical(cleared['trash'], categories=trash_cats)


#Вытаскиваем из колонки 'Дом' этаж квартиры, этажность и тип дома
cleared['floor'] = cleared['Дом'].str.split(',').str[0].str.split('/').str[0].astype('int8')
cleared['storeys'] = cleared['Дом'].str.split(',').str[0].str.split('/').str[1].astype('int8')
cleared['home_type'] = cleared['Дом'].str.split(',').str[1].fillna('unknown').str.strip().str.lower()

#Щитовых и деревянных домов всего 6 на весь датасет. По цифрам щитовой похож на панельный, а деревянный на блочный. Переназначим втупую
home_type_map = {'unknown':'unknown', 'деревянный':'блочный', 'блочный':'блочный', 'панельный':'панельный', 'щитовой':'панельный', 'кирпичный':'кирпичный', 'монолитный':'монолитный', 'сталинский':'сталинский', 'монолитно-кирпичный':'монолитно-кирпичный', 'старый фонд':'старый фонд'}
cleared['home_type'] = cleared['home_type'].map(home_type_map)

#Разобьём по категориям home_type
home_type_cats = ['старый фонд', 'сталинский', 'монолитно-кирпичный', 'блочный', 'unknown', 'монолитный', 'кирпичный', 'панельный']
cleared['home_type'] = pd.Categorical(cleared['home_type'], categories=home_type_cats)

#Добавим флаги на этажи
cleared['first_floor'] = cleared['floor'].eq(1).astype('int8')
cleared['last_floor'] = cleared['floor'].eq(cleared['storeys']).astype('int8')

#Сделаем флаг на наличие Названия ЖК в объявлении
cleared['residential_name'] = np.where(cleared['Название ЖК'].isna(), 0, 1).astype('int8')

# Обработка столбца Можно с детьми/животными
cleared['Можно с детьми/животными'] = cleared['Можно с детьми/животными'].fillna('unknown')
child_cats = ['Можно с детьми, Можно с животными','Можно с детьми','Можно с животными','unknown']
cleared['Можно с детьми/животными'] = pd.Categorical(cleared['Можно с детьми/животными'], categories=child_cats)

#Делаем флаги на то кто может проживать в квартире
cleared['unknown_allowed'] = cleared['Можно с детьми/животными'].eq('unknown').astype('int8')
cleared['childrens'] = cleared['Можно с детьми/животными'].isin(['Можно с детьми', 'Можно с детьми, Можно с животными']).astype('int8')
cleared['pets'] = cleared['Можно с детьми/животными'].isin(['Можно с животными', 'Можно с детьми, Можно с животными']).astype('int8')

# Обработка столбца Серии дома
# Создаем новый столбец: 1 если есть название, 0 если NaN
cleared['is_home_series'] = np.where(cleared['Серия дома'].isna(), 0, 1).astype('int8')

# Ячейка elevator извлекает количество лифтов в доме
# 0 - отсутствие значения
cleared['elevator'] = cleared['Лифт'].apply(resum).astype('int8')

# Ячейка balcony извлекает количество балконов в квартире
# 0 - отсутствие значения
cleared['balcony'] = cleared['Балкон'].apply(resum).astype('int8')

cleared = cleared.rename(columns={'ID  объявления': 'ID'})

In [ ]:
del cleared['Количество комнат']
del cleared['Парковка']
del cleared['Тип'] #везде одно значение, нет смысла от колонки
del cleared['Unnamed: 0'] #от индекса тоже пользы нет
del cleared['Ремонт']
del cleared['Мусоропровод']
del cleared['Дом']
del cleared['Площадь, м2']
del cleared['Цена']
del cleared['Площадь комнат, м2']
del cleared['Название ЖК']
del cleared['Можно с детьми/животными']
del cleared['Серия дома']
del cleared['Лифт']
del cleared['Санузел']
del cleared['Балкон']
del cleared['Ссылка на объявление']
del cleared['Окна']
del cleared['Телефоны']
del cleared['Метро']
del cleared['Адрес']
del cleared['Описание']
del cleared['Дополнительно']

In [ ]:
df2 = pd.read_csv("../data/data_DmitriiPrikhodko.csv")
del df2['Unnamed: 0']
for_merge = cleared.copy()

In [ ]:
merged=pd.merge(for_merge, df2, on='ID')

In [ ]:
#Кодируем категориальные колонки в цифры
cat_cols = merged.select_dtypes('category').columns.tolist()
team1_data = pd.get_dummies(merged, columns=cat_cols, dtype='int8')

In [ ]:
team1_data.to_csv('team1_data.csv', index=False, encoding='utf-8')